# 自動機械学習を使用して分類モデルをトレーニングする

モデルのトレーニングに使用できる機械学習アルゴリズムにはさまざまな種類があり、特定のデータや予測要件に最も効果的なアルゴリズムを特定するのは容易ではない場合があります。 さらに、正規化、欠落している機能の代入などの手法を使用して、トレーニング データを前処理することによってモデルの予測パフォーマンスに大きな影響を与えることができます。 要件に最適なモデルを見つけるには、多くのアルゴリズムと前処理変換の組み合わせを試す必要があります。これは多くの時間とコンピューティング リソースを必要とします。

Azure Machine Learning により、さまざまなアルゴリズムと前処理オプションを使用して、トレーニング済みモデルの比較を自動化できます。 [Azure Machine Learning スタジオ](https://ml/azure.com)または Python SDK (v2) のビジュアル インターフェイスを使って、この機能を活用することができます。 Python SDK では、自動機械学習ジョブの設定をより細かくコントロールできますが、より使いやすいのはビジュアル インターフェイスです。

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は、既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、スタジオの右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## データを準備する

自動機械学習用トレーニング スクリプトを作成する必要はありませんが、トレーニング データを読み込む必要があります。 

この場合は、糖尿病患者の詳細を含むデータセットを使用します。 

データセットを自動機械学習ジョブへの入力として渡すには、データを表形式で指定し、ターゲット列を含める必要があります。 データが表形式データセットとして解釈されるためには、入力データセットが **MLTable** である必要があります。

MLTable データ資産は、セットアップ中に既に作成されています。 **[データ]** ページに移動すると、データ資産を調べることができます。 ここでデータ資産を取得するには、その名前 `diabetes-training-table` とバージョン `1` を指定します。 

In [ ]:
## 自動機械学習ジョブを構成する

これで、自動機械学習実験を構成する準備が整いました。

次のコードを実行すると、次のような自動化された機械学習ジョブが作成されます。

- `aml-cluster` という名前のコンピューティング クラスターを使用する
- ターゲット列として `Diabetic` を設定する
- プライマリ メトリックとして `accuracy` を設定する
- 合計トレーニング時間が `60` 分後にタイムアウトする 
- 最大 `5` 個のモデルをトレーニングする
- モデルは `LogisticRegression` アルゴリズムを使用してトレーニングされない

In [ ]:
## 自動化された機械学習ジョブを実行する

準備完了です。 自動機械学習実験を実行しましょう。

> **注**: これには時間がかかる場合があります。

ジョブの実行中は、スタジオで監視できます。

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.